In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt

/home/nwertheim/Master-Thesis/envnoor/bin/python


In [2]:
data_file = os.path.join(univariate_data_path, 'merged_univariate.npy')
data = np.load(data_file, allow_pickle=True)
print(data.shape)
#test


# Check for NaN or Inf values in signals
for sample in data:
    if np.isnan(sample['signal']).any() or np.isinf(sample['signal']).any():
        print(f"NaN or Inf detected in {sample['record_name']}")

(666,)


In [3]:
def create_windows(sequence, window_size, step_size):
    windows = [sequence[i:i+window_size] for i in range(0, len(sequence) - window_size + 1, step_size)]
    return np.array(windows)

# Set windowing parameters
window_size = 500  
step_size = 250    

# Process each record and apply windowing
all_windows = [create_windows(record['signal'], window_size, step_size) for record in data]
all_windows = np.concatenate(all_windows, axis=0)

In [4]:
# Function to mask a percentage of data in each window
def mask_data(windows, mask_percentage=0.2):
    masked_windows = windows.copy()
    num_masked = int(mask_percentage * windows.shape[1])
    for i in range(windows.shape[0]):
        mask_indices = np.random.choice(windows.shape[1], num_masked, replace=False)
        masked_windows[i, mask_indices] = np.nan  
    return np.nan_to_num(masked_windows, nan=0.0)



In [5]:
# Code for final autoencoder
# import tensorflow as tf
# from tensorflow.keras import layers, models
# import matplotlib.pyplot as plt

# class MaskedAutoencoder(tf.keras.Model):
#     def __init__(self, input_dim):
#         super(MaskedAutoencoder, self).__init__()

#         # Encoder
#         self.encoder = models.Sequential([
#             layers.Dense(128),
#             layers.ReLU(),
#             layers.Dense(64),
#             layers.ReLU(),
#         ])

#         # Decoder
#         self.decoder = models.Sequential([
#             layers.Dense(128),
#             layers.ReLU(),
#             layers.Dense(input_dim, activation='linear')  # Reconstruct original input
#         ])

#     def call(self, inputs):
#         encoded = self.encoder(inputs)
#         decoded = self.decoder(encoded)
#         return decoded

# # Set input shape
# input_dim = 500  # Each window has 500 elements
# autoencoder = MaskedAutoencoder(input_dim)

# # Use Adam optimizer with lower learning rate and gradient clipping
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)

# autoencoder.compile(optimizer=optimizer, loss='mse')

# # Train the model and store history
# history = autoencoder.fit(masked_windows, all_windows, epochs=20, batch_size=64, validation_split=0.1)

# # Plot loss per epoch
# plt.figure(figsize=(8, 5))
# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss', linestyle='dashed')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Autoencoder Training Loss per Epoch')
# plt.legend()
# plt.grid()
# plt.show()
# # Save the encoder model


In [6]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# import matplotlib.pyplot as plt
# from skopt import gp_minimize
# from skopt.space import Real, Integer, Categorical
# from skopt.utils import use_named_args

# # Hyperparameter search space
# search_space = [
#     Real(0.1, 0.75, name='masking_ratio'),  # Masking ratio (10% - 75%)
#     Real(1e-5, 1e-2, "log-uniform", name='learning_rate'),  # Learning rate
#     Categorical([16, 32, 64], name='batch_size'),  # Only powers of 2
#     Integer(1, 5, name='num_layers'),  # Number of hidden layers (1 to 5)
#     Integer(32, 256, name='num_nodes')  # Number of nodes in each layer (32 to 256)
# ]

# class MaskedAutoencoder(tf.keras.Model):
#     def __init__(self, input_dim, num_layers, num_nodes):
#         super(MaskedAutoencoder, self).__init__()

#         # Encoder
#         self.encoder = models.Sequential()
#         for _ in range(num_layers):
#             self.encoder.add(layers.Dense(num_nodes, activation='relu'))

#         # Decoder
#         self.decoder = models.Sequential()
#         for _ in range(num_layers):
#             self.decoder.add(layers.Dense(num_nodes, activation='relu'))
#         self.decoder.add(layers.Dense(input_dim, activation='linear'))  # Reconstruct original input

#     def call(self, inputs):
#         encoded = self.encoder(inputs)
#         decoded = self.decoder(encoded)
#         return decoded

# # File to save results
# results_file = "bayesian_optimization_results.txt"
# plot_file = "bayesian_optimization_plot.png"

# def save_results(text):
#     with open(results_file, "a") as f:
#         f.write(text + "\n")

# # Optimization function
# @use_named_args(search_space)
# def objective(masking_ratio, learning_rate, batch_size, num_layers, num_nodes):
#     result_text = f"\nTesting Hyperparameter Combination:\n"
#     result_text += f" - Masking Ratio: {masking_ratio:.2f}\n"
#     result_text += f" - Learning Rate: {learning_rate:.5f}\n"
#     result_text += f" - Batch Size: {batch_size}\n"
#     result_text += f" - Number of Layers: {num_layers}\n"
#     result_text += f" - Number of Nodes: {num_nodes}\n"
#     print(result_text)
#     save_results(result_text)

#     # Apply masking
#     masked_windows = mask_data(all_windows, mask_percentage=masking_ratio)

#     # Build and compile model
#     autoencoder = MaskedAutoencoder(input_dim=window_size, num_layers=num_layers, num_nodes=num_nodes)
#     optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
#     autoencoder.compile(optimizer=optimizer, loss='mse')

#     # Train the model
#     history = autoencoder.fit(masked_windows, all_windows, epochs=10, batch_size=batch_size, validation_split=0.1, verbose=0)

#     final_loss = history.history['val_loss'][-1]
#     result_text = f" - Validation Loss: {final_loss:.5f}\n"
#     print(result_text)
#     save_results(result_text)

#     return final_loss

# # Run Bayesian Optimization
# result = gp_minimize(objective, search_space, n_calls=20, random_state=42)

# # Save and print best hyperparameters
# best_params_text = "\nBest Hyperparameters:\n"
# best_params_text += f"Masking Ratio: {result.x[0]:.2f}\n"
# best_params_text += f"Learning Rate: {result.x[1]:.5f}\n"
# best_params_text += f"Batch Size: {result.x[2]}\n"
# best_params_text += f"Number of Layers: {result.x[3]}\n"
# best_params_text += f"Number of Nodes: {result.x[4]}\n"
# print(best_params_text)
# save_results(best_params_text)

# # Plot optimization results
# plt.plot(result.func_vals)
# plt.xlabel("Iteration")
# plt.ylabel("Validation Loss (MSE)")
# plt.title("Bayesian Optimization Progress")
# plt.savefig(plot_file)  # Save the plot as an image
# plt.show()


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

# Hyperparameter search space
search_space = [
    Real(0.1, 0.75, name='masking_ratio'),
    Real(1e-5, 1e-2, "log-uniform", name='learning_rate'),
    Categorical([16, 32, 64], name='batch_size'),
    Integer(2, 5, name='num_layers'),  # Changed from 1-5 to 2-5
]

class MaskedAutoencoder(tf.keras.Model):
    def __init__(self, input_dim, num_layers, num_nodes_per_layer):
        super(MaskedAutoencoder, self).__init__()

        num_nodes_per_layer = list(num_nodes_per_layer)[-num_layers:]  # Ensure correct selection

        # Encoder (decreasing nodes per layer)
        self.encoder = models.Sequential()
        for i in range(num_layers):
            self.encoder.add(layers.Dense(num_nodes_per_layer[i], activation='relu'))

        # Decoder (increasing nodes per layer, mirroring encoder)
        self.decoder = models.Sequential()
        for i in range(num_layers - 1, -1, -1):
            self.decoder.add(layers.Dense(num_nodes_per_layer[i], activation='relu'))
        self.decoder.add(layers.Dense(input_dim, activation='linear'))  # Reconstruct original input

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

# File to save results
results_file = "bayesian_optimization_results.txt"
plot_file = "bayesian_optimization_plot.png"

def save_results(text):
    with open(results_file, "a") as f:
        f.write(text + "\n")

# Optimization function
@use_named_args(search_space)
def objective(masking_ratio, learning_rate, batch_size, num_layers):
    num_nodes_per_layer = []
    num_nodes = [32, 64, 128, 256, 512]
    for i in range(num_layers):
        num_nodes_per_layer.append(num_nodes[i]) 
    num_nodes_per_layer.reverse()

    # num_nodes_per_layer = list(num_nodes_per_layer)
    # if len(num_nodes_per_layer) < num_layers:
    #     num_nodes_per_layer = num_nodes_per_layer + [num_nodes_per_layer[-1]] * (num_layers - len(num_nodes_per_layer))


    result_text = f"\nTesting Hyperparameter Combination:\n"
    result_text += f" - Masking Ratio: {masking_ratio:.2f}\n"
    result_text += f" - Learning Rate: {learning_rate:.5f}\n"
    result_text += f" - Batch Size: {batch_size}\n"
    result_text += f" - Number of Layers: {num_layers}\n"
    result_text += f" - Number of Nodes Per Layer: {num_nodes_per_layer}\n"
    print(result_text)
    save_results(result_text)

    # Apply masking
    masked_windows = mask_data(all_windows, mask_percentage=masking_ratio)

    # Build and compile model
    autoencoder = MaskedAutoencoder(input_dim=window_size, num_layers=num_layers, num_nodes_per_layer=num_nodes_per_layer)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
    autoencoder.compile(optimizer=optimizer, loss='mse')

    # Train the model
    history = autoencoder.fit(masked_windows, all_windows, epochs=10, batch_size=batch_size, validation_split=0.1, verbose=0)

    final_loss = history.history['val_loss'][-1]
    result_text = f" - Validation Loss: {final_loss:.5f}\n"
    print(result_text)
    save_results(result_text)

    return final_loss

# Run Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=20, random_state=42)

# Save and print best hyperparameters
best_params_text = "\nBest Hyperparameters:\n"
best_params_text += f"Masking Ratio: {result.x[0]:.2f}\n"
best_params_text += f"Learning Rate: {result.x[1]:.5f}\n"
best_params_text += f"Batch Size: {result.x[2]}\n"
best_params_text += f"Number of Layers: {result.x[3]}\n"
best_params_text += f"Number of Nodes Per Layer: {list(result.x[4])[-result.x[3]:]}\n"
print(best_params_text)
save_results(best_params_text)

# Plot optimization results
plt.plot(result.func_vals)
plt.xlabel("Iteration")
plt.ylabel("Validation Loss (MSE)")
plt.title("Bayesian Optimization Progress")
plt.savefig(plot_file)  # Save the plot as an image
plt.show()



Testing Hyperparameter Combination:
 - Masking Ratio: 0.62
 - Learning Rate: 0.00004
 - Batch Size: 64
 - Number of Layers: 4
 - Number of Nodes Per Layer: [256, 128, 64, 32]



2025-03-25 17:35:53.772792: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


 - Validation Loss: 1.60723


Testing Hyperparameter Combination:
 - Masking Ratio: 0.39
 - Learning Rate: 0.00002
 - Batch Size: 32
 - Number of Layers: 3
 - Number of Nodes Per Layer: [128, 64, 32]

 - Validation Loss: 1.04106


Testing Hyperparameter Combination:
 - Masking Ratio: 0.19
 - Learning Rate: 0.00090
 - Batch Size: 16
 - Number of Layers: 4
 - Number of Nodes Per Layer: [256, 128, 64, 32]

 - Validation Loss: 0.44836


Testing Hyperparameter Combination:
 - Masking Ratio: 0.71
 - Learning Rate: 0.00001
 - Batch Size: 64
 - Number of Layers: 4
 - Number of Nodes Per Layer: [256, 128, 64, 32]

 - Validation Loss: 1.82547


Testing Hyperparameter Combination:
 - Masking Ratio: 0.50
 - Learning Rate: 0.00001
 - Batch Size: 16
 - Number of Layers: 4
 - Number of Nodes Per Layer: [256, 128, 64, 32]

 - Validation Loss: 1.50447


Testing Hyperparameter Combination:
 - Masking Ratio: 0.36
 - Learning Rate: 0.00001
 - Batch Size: 64
 - Number of Layers: 3
 - Number of Nodes Per La

IndexError: list index out of range

In [26]:
# path = os.path.join(models_path, 'encoder_model.keras')
# autoencoder.encoder.save(path)  # Saves in Keras format
# print('Saved encoder!')

NameError: name 'autoencoder' is not defined